# Extract Behavioral Data
Status: Implemented
 -- Fully automatized

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from utils.utils import *

In [2]:
# get all files from participants
folders = get_participant_folder_per_participant()
hdf_files = get_hdf_files_per_participant()
sequence_orders = get_sequence_order_per_participant()
assert (set(folders.keys()) == set(hdf_files.keys()))
assert (set(folders.keys()) == set(sequence_orders.keys()))

In [3]:
snippets_data = pd.read_csv(SNIPPETS_PATH, sep=SEPARATOR).fillna(0)
snippets_data[SNIPPET_COLUMN_SNIPPET] = snippets_data[SNIPPET_COLUMN_SNIPPET].str.replace(
    '\'', '')

### Get behavioral data
* store events for EEG annotations
* transform events into usable dataframe
* check sequence order

In [4]:
# Iterate over all participants
all_behavioral_data = []
all_behavioral_events = []

for participant in tqdm(folders):
    print('----------------------------------------------')
    print(PARTICIPANT, participant)

    event_data_files = hdf_files[participant]

    participant_behavioral_data = []
    participant_behavioral_events = []

    for i, hdf_file in tqdm(enumerate(event_data_files)):
        print(f'(01/10) Load behavioral events from file {hdf_file}')
        message_events = retrieve_message_events(hdf_file)
        if message_events.empty:
            print(f'\tNo behavioral events found in file {hdf_file}.')
            update_exclusions(participant, HDF_FILE, BEHAVIORAL, {
                hdf_file.name: '01c1_PRE_BEH_Event_Data: No behavioral events found in file.'})
            continue

        # Behavioral: get data according to categories (FIXATION)
        print('(02/10) Retrieve fixation cross data')
        snippet_fixation_events = get_fixation_cross_data(message_events)
        # display(snippet_fixation_events)
        if (message_events.shape[0] == 1) and (message_events[MESSAGE_COLUMN_CATEGORY] == MESSAGE_CATEGORY_FIXATION).all():
            print(f'\tOnly 1 fixation cross event found in file {hdf_file}.')
            behavioral_events = prepare_for_EEG_synchronization(
                snippet_fixation_events, None)
            behavioral_events[HDF_INDEX] = i
            participant_behavioral_events.append(behavioral_events)
            update_exclusions(participant, HDF_FILE, BEHAVIORAL, {
                hdf_file.name: '01c1_PRE_BEH_Event_Data: No behavioral events found in file.'})
            continue

        # Behavioral: get data according to categories (SNIPPETVIEW for start and end snippet-view => answer time)
        print('(03/10) Retrieve / calculate timing data')
        snippet_duration_events = get_snippet_view_data(
            message_events, snippet_fixation_events)
        # display(snippet_duration_events)

        # Behavioral: get data according to categories (ANSWER for correctness)
        print('(04/10) Retrieve answer data')
        snippet_answer_events = get_snippet_answer_data(message_events)
        # display(snippet_answer_events)

        # Behavioral: get data according to categories (PERCEPTION for subjective difficulty rating)
        print('(05/10) Retrieve perception / rating data')
        snippet_perception_events = get_snippet_perception_data(message_events)
        # display(snippet_perception_events)

        print('(06/10) Transform snippets data')
        # merge all information retrieved from events
        behavioral_data = transform_snippet_data(
            snippet_fixation_events, snippet_duration_events, snippet_answer_events, snippet_perception_events, snippets_data)

        print('(07/10) Prepare for concat of hdf file results')
        # delete unnecessary columns
        behavioral_data = behavioral_data.drop(columns=[SNIPPET_COLUMN_SNIPPET]+[
                                               c for c in behavioral_data.columns if not (c in BEHAVIORAL_COLUMNS)])
        # add information
        behavioral_data[HDF_INDEX] = i
        behavioral_data[PARTICIPANT] = participant
        participant_behavioral_data.append(behavioral_data)
        # display(behavioral_data)

        print('(08/10) Prepare snippet events for eeg trial assignment')
        # linear events and corresponding stimuli code
        behavioral_events = prepare_for_EEG_synchronization(
            snippet_fixation_events, snippet_duration_events)
        behavioral_events[HDF_INDEX] = i
        participant_behavioral_events.append(behavioral_events)

    # add extracted snippets to exclusion file - respective after all files delete those that did not occur
    print('(09/10) Check that sequence order is retained and complete')
    behavioral_data = pd.concat([bd.sort_values(
        [BEHAVIORAL_COLUMN_START, BEHAVIORAL_COLUMN_END]) for bd in participant_behavioral_data]).reset_index(drop=True)
    behavioral_data = compare_behavioral_to_sequence_order(participant, behavioral_data, sequence_orders[participant],
                                                           '01c1_PRE_BEH_Event_Data')
    # display(behavioral_data)

    print('(10/10) Save participant behavioral data')
    # save behavioral data
    behavioral_data.to_csv(get_behavioral_data_path(
        participant, final_data_exclusion=False), index=False, sep=SEPARATOR)
    behavioral_events = pd.concat(participant_behavioral_events)
    behavioral_events[PARTICIPANT] = participant
    behavioral_events.to_csv(get_behavioral_events_path(participant),
                             index=False, sep=SEPARATOR)
    all_behavioral_data.append(behavioral_data)
    all_behavioral_events.append(behavioral_events)

print('(10/10) Save all behavioral data')
# save behavioral data
behavioral_data = pd.concat(all_behavioral_data)
behavioral_data.to_csv(get_behavioral_data_path(
    final_data_exclusion=False), index=False, sep=SEPARATOR)
behavioral_events = pd.concat(all_behavioral_events)
behavioral_events[PARTICIPANT] = participant
behavioral_events.to_csv(get_behavioral_events_path(),
                         index=False, sep=SEPARATOR)

  0%|          | 0/24 [00:00<?, ?it/s]

----------------------------------------------
Participant 003


0it [00:00, ?it/s]

(01/10) Load behavioral events from file ..\08-Data-Trial_Recordings\raw\E003\Experiment_003_part01.hdf5
(02/10) Retrieve fixation cross data
(03/10) Retrieve / calculate timing data
(04/10) Retrieve answer data
(05/10) Retrieve perception / rating data
(06/10) Transform snippets data
(07/10) Prepare for concat of hdf file results
(08/10) Prepare snippet events for eeg trial assignment
(09/10) Check that sequence order is retained and complete
(10/10) Save participant behavioral data
----------------------------------------------
Participant 004


0it [00:00, ?it/s]

...

(01/10) Load behavioral events from file ..\08-Data-Trial_Recordings\raw\E101\Experiment_101_part01.hdf5
(02/10) Retrieve fixation cross data
(03/10) Retrieve / calculate timing data
(04/10) Retrieve answer data
(05/10) Retrieve perception / rating data
(06/10) Transform snippets data
(07/10) Prepare for concat of hdf file results
(08/10) Prepare snippet events for eeg trial assignment
(09/10) Check that sequence order is retained and complete
(10/10) Save participant behavioral data
----------------------------------------------
Participant 102


0it [00:00, ?it/s]

(01/10) Load behavioral events from file ..\08-Data-Trial_Recordings\raw\E102\Experiment_102_part01.hdf5
(02/10) Retrieve fixation cross data
(03/10) Retrieve / calculate timing data
(04/10) Retrieve answer data
(05/10) Retrieve perception / rating data
(06/10) Transform snippets data
(07/10) Prepare for concat of hdf file results
(08/10) Prepare snippet events for eeg trial assignment
(01/10) Load behavioral events from file ..\08-Data-Trial_Recordings\raw\E102\Experiment_102_part02.hdf5
(02/10) Retrieve fixation cross data
(03/10) Retrieve / calculate timing data
(04/10) Retrieve answer data
(05/10) Retrieve perception / rating data
(06/10) Transform snippets data
(07/10) Prepare for concat of hdf file results
(08/10) Prepare snippet events for eeg trial assignment
(09/10) Check that sequence order is retained and complete
(10/10) Save participant behavioral data
----------------------------------------------
Participant 106


0it [00:00, ?it/s]

(01/10) Load behavioral events from file ..\08-Data-Trial_Recordings\raw\E106\Experiment_106_part01.hdf5
(02/10) Retrieve fixation cross data
	Only 1 fixation cross event found in file ..\08-Data-Trial_Recordings\raw\E106\Experiment_106_part01.hdf5.
(01/10) Load behavioral events from file ..\08-Data-Trial_Recordings\raw\E106\Experiment_106_part02.hdf5
(02/10) Retrieve fixation cross data
	Only 1 fixation cross event found in file ..\08-Data-Trial_Recordings\raw\E106\Experiment_106_part02.hdf5.
(01/10) Load behavioral events from file ..\08-Data-Trial_Recordings\raw\E106\Experiment_106_part03.hdf5
(02/10) Retrieve fixation cross data
	Only 1 fixation cross event found in file ..\08-Data-Trial_Recordings\raw\E106\Experiment_106_part03.hdf5.
(01/10) Load behavioral events from file ..\08-Data-Trial_Recordings\raw\E106\Experiment_106_part04.hdf5
(02/10) Retrieve fixation cross data
	Only 1 fixation cross event found in file ..\08-Data-Trial_Recordings\raw\E106\Experiment_106_part04.hdf5.
